In [1]:
memory.limit()
memory.size()
gc()

[1] 32698

[1] 47.22

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,510170,27.3,1146672,61.3,621911,33.3
Vcells,941585,7.2,8388608,64.0,1603065,12.3


In [ ]:
library(tidyverse, quietly = TRUE)

metrics_df <- read_csv("E:\\work\\results\\grid_search\\metrics.csv")
iou_df <- metrics_df[metrics_df$metric == 'iou', ]
which(! complete.cases(iou_df))

test_metrics_df <- iou_df[iou_df$type == 'test', ]
which(! complete.cases(test_metrics_df))
val_metrics_df <- iou_df[iou_df$type == 'val', ]
which(! complete.cases(val_metrics_df))
nrow(unique(iou_df[c("job_hash")]))

head(test_metrics_df)

Parsed with column specification:
cols(
  image = col_character(),
  class = col_character(),
  threshold = col_double(),
  metric = col_character(),
  type = col_character(),
  value = col_double(),
  data_dataset = col_character(),
  model_activation = col_character(),
  model_depth = col_double(),
  model_in_channels = col_double(),
  model_out_classes = col_double(),
  model_residual_units = col_double(),
  model_width = col_double(),
  model_width_ratio = col_double(),
  job_hash = col_character(),
  epoch = col_double(),
  loss = col_double(),
  predict_time = col_double()
)


**Test Dataset**

In [ ]:
which(! complete.cases(test_metrics_df))
model <- aov(value~job_hash, data=test_metrics_df)
summary(model)
tukey <- TukeyHSD(model, conf.level=.95, ordered=TRUE)
results <- as.data.frame(tukey$job_hash)
names(results) <- c("diff", "lwr", "upr", "p_adj")
results <- results[order(results$diff, decreasing=TRUE), ] 

In [ ]:
write.csv(results,"E:\\work\\results\\grid_search\\test_tukey.csv", row.names = TRUE)
length( which( results$p_adj > 0.05 ) )
length( which( results$p_adj < 0.05 ) )

In [ ]:
best_job_hashes <- c(
    'bfd33d38ce9c4d84b912f1ee0b7961aa',
    '3d0045f175d096e795e02af87c91c5fc',
    '117d2454d163eb9fc94ae1fac18b036d',
    'e1873349eb16479c563b4f3690a5b162',
    '7f1a84913968ff6bc745dc21d0088207',
    '645eb73f1c112a5a17e7fdbfb3c1b0cb',
    'e310e988831119b2425954ce5d38f040',
    '0f0e7bd0220a6492eefad6318cc159d1',
    '2d844100ab842f7cd5142ab5284c83b3',
    '5d22427ae45da60a71182bdd1ce8df4c',
    '791687cd35791695e49a0be3f0071702',
    '437a4dd2569fabc9367fa619ef599dae',
    '0c9b3e43ee7d293529e9c5da1cb72ce4',
    '4218d972fc50a0fcb380d26d9a44bef9',
    '6e8912b429699396a12723b2bcaeff78',
    '1ec1b3cdde28b001830e9d07ac190d3d'
)

results[c('better', 'worse')] <- str_split_fixed(rownames(results), '-', 2)
best_results <- results[results$best %in% best_job_hashes, ]
best_results <- results[results$worse %in% best_job_hashes, ]

best_metrics <- test_metrics_df[test_metrics_df$job_hash %in% best_job_hashes, ]

 p <- ggboxplot(best_metrics, x = "job_hash", y = "value",
                color = "job_hash",
                add = "jitter", shape = "job_hash")

**Val Dataset**

In [ ]:
model <- aov(value~job_hash, data=val_metrics_df)
summary(model)
tukey <- TukeyHSD(model, conf.level=.95, ordered=TRUE)
results <- as.data.frame(tukey$job_hash)
names(results) <- c("diff", "lwr", "upr", "p_adj")
results <- results[order(results$diff, decreasing=TRUE), ] 
write.csv(results,"E:\\work\\results\\grid_search\\val_tukey.csv", row.names = TRUE)
length( which( results$p_adj > 0.05 ) )
length( which( results$p_adj < 0.05 ) )

**Best Models**

In [ ]:
model <- aov(value~job_hash, data=best_metrics_df)
summary(model)
tukey <- TukeyHSD(model, conf.level=.95, ordered=TRUE)
plot(tukey)
results <- as.data.frame(tukey$job_hash)
names(results) <- c("diff", "lwr", "upr", "p_adj")
results <- results[order(results$diff, decreasing=TRUE), ] 
write.csv(results,"E:\\work\\results\\grid_search\\test_tukey_best.csv", row.names = TRUE)
length( which( results$p_adj > 0.05 ) )
length( which( results$p_adj < 0.05 ) )